# 基于物品的协同过滤算法 ItemCF

In [1]:
# 导入包
import random
import math
import time
from tqdm import tqdm

## 一. 通用函数定义

In [2]:
# 定义装饰器，监控运行时间
def timmer(func):
    def wrapper(*arg, **kwargs):
        start_time = time.perf_counter()
        res = func(*arg, **kwargs)
        stop_time = time.perf_counter()
        print("Func %s, run time: %s" % (func.__name__, stop_time - start_time))
        return res
    return wrapper

### 1. 数据处理相关
- load data
- split data

数据格式（有换行）： `[user_id ::  movie_id :: rating :: timestamp]`

        1::1193::5::978300760

        1::661::3::978302109
        
- 基于用户或item的协同过滤只需要保留<font color=red> user </font>和 <font color=red> item </font>字段数据即可。处理数据时，需要将换行符 `'\n'`和 分隔符`'::'`去掉;
- 需求数据格式 tuple(user, item), tuple元素为int

In [3]:
class Dataset():
    def __init__(self, filepath):
        self.data = self.loadData(filepath)
        
    @timmer
    def loadData(self, filepath):
        '''返回值data: 元组(user, item)为元素的列表'''
        data = []
        for line in open(filepath):
            temp = tuple(map(int, line.strip().split("::")[:2]))   # index 0和1是 user和item
            data.append(temp)
#         print('部分原始数据: {}'.format(data[:10]), type(data))
        return data
    
    @timmer
    def splitData(self, kfolds=5, seed=1):
        '''
        :params: data, 格式元组(user, item)
        :params: kfolds, k折交叉验证的n_split 
        :params: seed, random的种子数
        '''
        train, test = [], []
        random.seed(seed)
        
        for user, item in self.data:
            # 这里与书中的不一致，本人认为取M-1较为合理，因randint是左右都覆盖的
            if random.randint(0, kfolds-1) == 1:  
                test.append((user, item))
            else:
                train.append((user, item))
                
#         print(len(train), len(test))
#         print(train[5800:5810], test[100:110])
            
        def convert_dict(data):
            '''转换成字典形式 {user: set(items)}'''
            data_dict = {}
            for user, item in data:
                if user not in data_dict:
                    data_dict[user] = set()
                data_dict[user].add(item)     # 集合添加元素的方法 .add(ele)
#             data_dict转化为 user 为key, set(item)转为 list 为value的字典（可要可不要）
#             for key in data_dict.keys():
#                 data_dict[key] = list(data_dict[key])
#             print('~~~~~~~~~~~~', data_dict)
            return data_dict
        return convert_dict(train), convert_dict(test)       

In [4]:
if __name__ == "__main__":
    filepath = r'..\data\ml-1m\ratings.dat'
    dataset = Dataset(filepath)
    dataset.splitData(5, 2)
    

Func loadData, run time: 1.5089765009999998
Func splitData, run time: 2.072656234


### 2. 评价指标
1. Precision
2. Recall
3. Coverage
4. Popularity(Novelty)

In [5]:
class Metric():
    def __init__(self, train, test, GetRecommendation):
        '''
        :params: train, 训练数据
        :params: test, 测试数据
        :params: GetRecommendation, 为某个用户获取推荐物品的接口函数
        '''
        self.train = train
        self.test = test
        self.GetRecommendation = GetRecommendation
        self.recs = self.getRec()
        
    def getRec(self):
        '''为 test 中的每个用户进行推荐'''
        recs = {}
        for user in self.test.keys():
            rank = self.GetRecommendation(user)  # rank为列表
            recs[user] = rank
        return recs
    
    def precision(self):
        '''准确率：最终的推荐列表中有多少比例是 **发生过的用户-物品行为记录** '''
        All, hit = 0, 0
        for user in self.test.keys():
            # 用户在test中喜欢的item 集合 T(u)
            test_items = set(self.test[user])
            # 对用户推荐的N个item 列表 （rank为列表）
            rank = self.recs[user]
            for item, score in rank:
                if item in test_items:
                    hit += 1               # 分子： sum(T(u) & R(u)))
            All += len(rank)               # 分母： sum(R(u))
            # precision = (T(u) & R(u)) / R(u)
        return round(hit / All * 100, 2)
        
    def recall(self):
        '''召回率：有多少比例的 **用户-物品行为记录** 包含在最终的推荐列表中'''
        All, hit = 0, 0
        for user in self.test.keys():
            # 用户在 test 中喜欢的 item 集合 T(u)
            test_items = set(self.test[user])
            # 对用户推荐的N个 item 列表R(u)
            rank = self.recs[user]
            for item, score in rank:
                if item in test_items:
                    hit += 1               # 分子： sum(T(u) & R(u)))
            All += len(test_items)         # 分母： sum(T(u))
            # recall = (T(u) & R(u)) / T(u)
        return round(hit / All * 100, 2)
    
    def coverage(self):
        '''覆盖率：最终的推荐列表中包含多大比例的 **物品**'''
        all_item, recom_item = set(), set()
        for user in self.test.keys():         # test 中的 user
            for item in self.train[user]:     # test中user在train中包含的所有item
                # 凡是train中user有过行为记录的item都加入到all_item中
                all_item.add(item)            # 所有物品集合 I
            # 对用户推荐的N个item 列表
            rank = self.recs[user]
            # 凡是推荐给user的item都计入到recom_item中
            for item, score in rank:
                recom_item.add(item)          # 推荐的物品集合 R(u)
        # coverage = #R(u) / #I
        return round(len(recom_item) / len(all_item) * 100, 2)
    
    def popularity(self):
        '''新颖度：推荐的是目标用户喜欢的但未发生过的用户-行为
           用推荐列表中物品的平均流行度来度量新颖度(新颖度越高，流行度越低)'''
        # 计算item 的流行度 （train set）
        item_popularity = {}
        for user, items in self.train.items():
            for item in items:
                if item not in item_popularity:
                    item_popularity[item] = 0
                # 若item在train的user中发生过记录，则该item的流行度+1
                item_popularity[item] += 1
        
        popular = 0
        num = 0
        for user in self.test.keys():
            # 向test中的 user 推荐topN 物品
            rank = self.recs[user]
            for item, score in rank:
                # 对每个物品的流行度取对数运算
                # 防止长尾问题带来的北流行物品主导的推荐（避免热门推荐）
                popular += math.log(1 + item_popularity[item])
                # 汇总所有user的总推荐物品个数
                num += 1
        # 计算平均流行度 = popular / n
        return round(popular / num, 6)
    
    def eval(self):
        # 汇总 metric 指标
        metric = {'Precision': self.precision(),
                  'Recall': self.recall(),
                  'Coverage': self.coverage(),
                  'Popularity': self.popularity()}
        print('Metric: {}'.format(metric))
        return metric

## 二、算法实现
- **ItermCF**
- **TtemIUF**
- **ItemCF_normalization**

### 1. ItemCF
- 基于物品的协同过滤
- 步骤：

    1. 建立 用户--物品倒排表 `user --> item`
        - 每个用户建立一个包含他喜欢的物品的列表
        - (该数据集存储格式就是 user-->set(items)倒排表格式)
    2. 物品两两共现的稀疏矩阵`Cmat[i][j]` -- (可用dict保存如UserCF)
        - 对每个用户，将他物品列表中的物品两两在共现矩阵中加1
        - `Cmat[i][j]`表示同时喜欢物品`i`和物品`j`的用户数
    3. 根据Cmat计算物品之间的余弦相似度矩阵
    4. 根据余弦相似度矩阵排序（降序），获得用户的物品列表中每一种`item`所对应的top-N个`items`
    

In [6]:
def ItemCF(train, K, N):
    '''
    :params: train, 训练数据集
    :params: K, 超参数，设置取TopK相似物品数目
    :params: N, 超参数，设置取TopN推荐物品数目
    :return: GetRecommendation, 推荐接口函数
    '''
    # 计算 items--items 的稀疏矩阵
    cmat = {}        # 两两item同时出现的次数 #(item1, item2). {item1:{item1,item2,item3,..}, item2:{item1,item2,item3,..},...}
    num = {}         # 单一item出现的次数 #item1  (相当于行索引的item出现的次数)
    for user in train.keys():
        items = train[user]
        for item1 in items:
            if item1 not in num.keys():  # 单一item出现的次数
                num[item1] = 0
            num[item1] += 1
            if item1 not in cmat.keys():
                cmat[item1] = {}
            for item2 in items:
                if item2 == item1:
                    continue
                if item2 not in cmat[item1]:
                    cmat[item1][item2] = 0
                cmat[item1][item2] += 1
                    
    # 计算余弦相似度
    sim = {}          # 初始化 相似度矩阵
    for i in cmat.keys():
        sim[i] = {}   # 初始化 sim[i]，确保sim[i]也是dict
        for j, cij in cmat[i].items():
            sim[i][j] = cij / math.sqrt(num[i] * num[j])

    # 按照相似度的值对矩阵的每一行进行降序排序
    sim_item_sorted = {}
    for key, values in sim.items():
        sim_item_sorted[key] = sorted(values.items(), key=lambda x: x[1], reverse=True)[:K]
        # sorted函数返回的是列表 list
#         print(sim_item_sorted[key], '--------------')
    

    # 为待推荐的用户获取推荐接口函数
    def GetRecommendation(user):
        rank = {}                                   # 待推荐列表  {item1:rank1, item2:rank2,...}
        interacted_items = set(train[user])         # 用户见过的item列表 [item1, item2, item3, ...]
        # 根据相似度高的用户的列表对user进行推荐（去掉user见过的item）
        for item in train[user]:                    # 遍历user的物品列表
            for i, _ in sim_item_sorted[item]:            # 与排序后的topK个相似物品进行相似度计算
                if i not in interacted_items:       # topK中的item不在user的已有列表中
                    if i not in rank.keys():        # topK中的item不在待推荐列表中
                        rank[i] = 0              # 不在rank中则添加进去并赋初值为0
                    rank[i] += sim[item][i]      # topK与用户已有items的两两共现余弦相似度矩阵
        # 对rank字典排序，获得 topN 对应的 item
        rank_sorted = sorted(rank.items(), key=lambda x: x[1], reverse=True)[:N]
        '''若只保存 item，不需要#item: [item1, item2, item3, ...]'''
#         rank_sorted = list(map(lambda x: x[0], rank_sorted))
        # 返回值是列表
        return rank_sorted
    
    return GetRecommendation

### 2. ItemIUF: 改进版ItemCF
- 考虑了活跃用户对物品相似度的贡献应该小于不活跃的用户
- 只改变了 相似度的计算公式

In [7]:
def ItemIUF(train, K, N):
    '''
    :params: train, 训练数据集
    :params: K, 超参数，设置取TopK相似物品数目
    :params: N, 超参数，设置取TopN推荐物品数目
    :return: GetRecommendation, 推荐接口函数
    '''
    # 计算 items--items 的稀疏矩阵
    cmat = {}        # 两两item同时出现的次数 #(item1, item2). {item1:{item1,item2,item3,..}, item2:{item1,item2,item3,..},...}
    num = {}         # 单一item出现的次数 #item1  (相当于行索引的item出现的次数)
    for user in train.keys():
        items = train[user]
        for item1 in items:
            if item1 not in num.keys():  # 单一item出现的次数
                num[item1] = 0
            num[item1] += 1
            if item1 not in cmat.keys():
                cmat[item1] = {}
            for item2 in items:
                if item2 == item1:
                    continue
                if item2 not in cmat[item1]:
                    cmat[item1][item2] = 0
                cmat[item1][item2] += 1 / math.log(1 + len(items))
                    
    # 计算余弦相似度
    sim = {}          # 初始化 相似度矩阵
    for i in cmat.keys():
        sim[i] = {}   # 初始化 sim[i]，确保sim[i]也是dict
        for j, cij in cmat[i].items():
            sim[i][j] = cij / math.sqrt(num[i] * num[j])

    # 按照相似度的值对矩阵的每一行进行降序排序
    sim_item_sorted = {}
    for key, values in sim.items():
        sim_item_sorted[key] = sorted(values.items(), key=lambda x: x[1], reverse=True)[:K]
        # sorted函数返回的是列表 list

    # 为待推荐的用户获取推荐接口函数
    def GetRecommendation(user):
        rank = {}                                   # 待推荐列表  {item1:rank1, item2:rank2,...}
        interacted_items = set(train[user])         # 用户见过的item列表 [item1, item2, item3, ...]
        # 根据相似度高的用户的列表对user进行推荐（去掉user见过的item）
        for item in train[user]:                    # 遍历user的物品列表
            for i, _ in sim_item_sorted[item]:            # 与排序后的topK个相似物品进行相似度计算
                if i not in interacted_items:       # topK中的item不在user的已有列表中
                    if i not in rank.keys():        # topK中的item不在待推荐列表中
                        rank[i] = 0              # 不在rank中则添加进去并赋初值为0
                    rank[i] += sim[item][i]      # topK与用户已有items的两两共现余弦相似度矩阵
        # 对rank字典排序，获得 topN 对应的 item
        rank_sorted = sorted(rank.items(), key=lambda x: x[1], reverse=True)[:N]
        '''若只保存 item，不需要#item: [item1, item2, item3, ...]'''
#         rank_sorted = list(map(lambda x: x[0], rank_sorted))
        # 返回值是列表
        return rank_sorted
    
    return GetRecommendation

### 3. ItemCF_Normalized  (相似度归一化后的ItemCF)
- 公式：` W(i,j) / max(W(i,j): j)`
- 归一化原因：
    - 避免了相似度基准不同的两个类对推荐的影响
    - 假设A类物品之间相似度为0.6， B类物品之间是0.5， A类与B类物品之间相似度是0.2, 则如果一共用户喜欢了5个A类和5个B类物品，用ItemCF推荐的就是A类物品，因为A类物品之间的相似度大，权重更高。
    - 相似度的归一化可以提高推荐的多样性，减弱热门推荐的趋势

In [8]:
def ItemCF_Norm(train, K, N):
    '''
    :params: train, 训练数据集
    :params: K, 超参数，设置取TopK相似物品数目
    :params: N, 超参数，设置取TopN推荐物品数目
    :return: GetRecommendation, 推荐接口函数
    '''
    # 计算 items--items 的稀疏矩阵
    cmat = {}        # 两两item同时出现的次数 #(item1, item2). {item1:{item1,item2,item3,..}, item2:{item1,item2,item3,..},...}
    num = {}         # 单一item出现的次数 #item1  (相当于行索引的item出现的次数)
    for user in train.keys():
        items = train[user]
        for item1 in items:
            if item1 not in num.keys():  # 单一item出现的次数
                num[item1] = 0
            num[item1] += 1
            if item1 not in cmat.keys():
                cmat[item1] = {}
            for item2 in items:
                if item2 == item1:
                    continue
                if item2 not in cmat[item1]:
                    cmat[item1][item2] = 0
                cmat[item1][item2] += 1 / math.log(1 + len(items))

    # 计算余弦相似度
    sim = {}          # 初始化 相似度矩阵
    for i in cmat.keys():
        sim[i] = {}   # 初始化 sim[i]，确保sim[i]也是dict
        for j, cij in cmat[i].items():
            sim[i][j] = cij / math.sqrt(num[i] * num[j])

    '''相似度矩阵归一化'''
    for i in sim.keys():
        s_max = 0                # 每遍历一行之前先初始化s_max为0
        for j in sim[i].keys():
            if sim[i][j] >= s_max:
                s_max = sim[i][j]
            sim[i][j] /= s_max
            
            
    # 按照相似度的值对矩阵的每一行进行降序排序
    sim_item_sorted = {}
    for key, values in sim.items():
        sim_item_sorted[key] = sorted(values.items(), key=lambda x: x[1], reverse=True)[:K]
        # sorted函数返回的是列表 list

    # 为待推荐的用户获取推荐接口函数
    def GetRecommendation(user):
        rank = {}                                   # 待推荐列表  {item1:rank1, item2:rank2,...}
        interacted_items = set(train[user])         # 用户见过的item列表 [item1, item2, item3, ...]
        # 根据相似度高的用户的列表对user进行推荐（去掉user见过的item）
        for item in train[user]:                    # 遍历user的物品列表
            for i, _ in sim_item_sorted[item]:              # 与排序后的topK个相似物品进行相似度计算
                if i not in interacted_items:       # topK中的item不在user的已有列表中
                    if i not in rank.keys():        # topK中的item不在待推荐列表中
                        rank[i] = 0              # 不在rank中则添加进去并赋初值为0
                    rank[i] += sim[item][i]      # topK与用户已有items的两两共现余弦相似度矩阵
        # 对rank字典排序，获得 topN 对应的 item
        rank_sorted = sorted(rank.items(), key=lambda x: x[1], reverse=True)[:N]
        '''若只保存 item，不需要#item: [item1, item2, item3, ...]'''
#         rank_sorted = list(map(lambda x: x[0], rank_sorted))
        # 返回值是列表
        return rank_sorted
    
    return GetRecommendation

## 三. 测试算法
1. ItemCF实验，K=[5, 10, 20, 40, 80, 160]
2. ItemIUF实验, K=10
3. ItemCF-Norm实验，K=10

    - K: topK相似物品数
    - N = 10 （top10作为推荐物品）

In [9]:
class Experiment():
    def __init__(self, K, N, fp=r'..\data\ml-1m\ratings.dat', method='ItemCF'):
        '''
        :params: K, TopK相似物品的个数
        :params: N, TopN推荐物品的个数
        :params: fp, 数据文件路径
        :params: method, 推荐算法
        '''
        self.K = K
        self.N = N
        self.fp = fp
        self.method = method
        self.alg = {"ItemCF": ItemCF, "ItemIUF": ItemIUF, "ItemCF_Norm": ItemCF_Norm}
        
    @timmer
    def worker(self, train, test):
        '''
        :params: train, 训练数据集
        :params: test, 测试数据集
        :return: 各指标的值
        '''
        getRecommendation = self.alg[self.method](train, self.K, self.N)
        metric = Metric(train, test, getRecommendation)
        return metric.eval()
    
    @timmer
    def run(self):
        dataset = Dataset(self.fp)
        train, test = dataset.splitData()
        metric = self.worker(train, test)
        print('Done!!') 

In [10]:
N = 10    # top10推荐物品

In [11]:
# 1. ItemCF
for K in [5, 10, 20, 40, 80, 160]:
    cf_exp = Experiment(K, N, method='ItemCF')
    cf_exp.run()

Func loadData, run time: 1.8424046480000005
Func splitData, run time: 2.147140878000001
Metric: {'Precision': 0.0, 'Recall': 0.0, 'Coverage': 29.13, 'Popularity': 7.241342}
Func worker, run time: 109.73245773800001
Done!!
Func run, run time: 113.83392251299999
Func loadData, run time: 1.744566539999994
Func splitData, run time: 2.0631984539999877
Metric: {'Precision': 0.0, 'Recall': 0.0, 'Coverage': 28.75, 'Popularity': 7.30798}
Func worker, run time: 108.82248727400001
Done!!
Func run, run time: 112.75671857999998
Func loadData, run time: 1.8838977089999958
Func splitData, run time: 2.0849937299999795
Metric: {'Precision': 0.0, 'Recall': 0.0, 'Coverage': 27.83, 'Popularity': 7.341531}
Func worker, run time: 115.76465060000001
Done!!
Func run, run time: 119.85137887300002
Func loadData, run time: 1.7169111380000004
Func splitData, run time: 2.0427394970000137
Metric: {'Precision': 0.0, 'Recall': 0.0, 'Coverage': 26.58, 'Popularity': 7.343813}
Func worker, run time: 121.53786298
Done!!


In [12]:
# 2. ItemIUF
K, N = 10, 10
cf_exp = Experiment(K, N, method='ItemIUF')
cf_exp.run()

Func loadData, run time: 1.8035546469999417
Func splitData, run time: 2.038005202000022
Metric: {'Precision': 0.0, 'Recall': 0.0, 'Coverage': 25.82, 'Popularity': 7.366785}
Func worker, run time: 226.823927701
Done!!
Func run, run time: 230.79605327500008


In [ ]:
K, N = 10, 10
cf_exp = Experiment(K, N, method='ItemCF_Norm')
cf_exp.run()

Func loadData, run time: 1.8150143469999875
Func splitData, run time: 2.0954095199999756
